# Experimento 4
Después de intentar usar gensim para generar un modelo usando redes neuronales, opté por realizar una propuesta más intuitiva, que obedece el siguiente diagrama:


![title](diagrama.png)

Por pasos tenemos el siguiente código, primero necesitamos una función que lea el corpus:

In [1]:
def leer(corpus):
	with open(corpus,"r") as f:
		for line in f:
			yield line.split("|",1)


Después dado que TfidfVectorizer regresa una matriz con los valores de tf-idf, necesitamos extraer el vocabulario y el valor tf-idf asociado, con el siguiente código:

In [2]:
def getData(coo_matrix,feature_names):
	puntos=[]
	nombres=[]
	for score,idx in zip(coo_matrix.data,coo_matrix.col):
		aux=feature_names[idx].rstrip('sao') 
		if aux not in nombres:
			puntos.append(score)
			nombres.append(aux)
		elif score>puntos[nombres.index(aux)]: #aqui podemos ver que tomo el tf más grande de todos
            #principal diferencia con el experimento 5
			puntos[nombres.index(aux)]=score
	tup=zip(nombres,puntos)
	return sorted(tup,key=lambda x: x[1],reverse=True)

Una vez que tenemos los valores tf-idf, es necesario calcular los vectores de cada documento, con el siguiente código

In [3]:
import unicodedata
import numpy as np
            
def doc2Vec(doc,vocab,funcion):
	palabras=list(map(lambda x: unicodedata.normalize('NFKD',x).rstrip('sao'),doc.split()))
	vector=[]
	for key,value in vocab.items():
		vector.append(funcion(palabras.count(key))*value)
	return np.array(vector).reshape(1,-1)

Luego debemos de tener una función que calcule los vectores de todos los documentos e integre las funciones anteriores, ésta es la siguiente: 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import time
from math import exp

def crearVectores(archCorpus,archPickle,funcion):
    # se usa un set de palabras que deben detenerse, lo encontré en este repo.
    # https://github.com/stopwords-iso/stopwords-es
	with open("../Experimento3/stopwords-es-master/stopwords-es.txt","r") as f:
	    stopwords=f.read().splitlines() 
	corpus=list(leer(archCorpus)) # leemos el corpus
	vectorizador=TfidfVectorizer(smooth_idf=True, #aplicamos un suavizado del idf, no afecta mucho al resultado
                                 use_idf=True, # usamos idf
                                 stop_words=stopwords, # le damos el set de palabras a deterner
                                 min_df=2, # queremos que las palabras aparezcan al menos en dos documentos
                                 #si cambiamos esto a 1, pone muchas palabras que solo hacen el programa mas lento
                                 strip_accents='ascii') # le aplica una normalización a las palabras
	palabras=vectorizador.fit_transform(map(lambda x:x[1],corpus)) #aqui se aplica el tf-idf
	feature_names=vectorizador.get_feature_names() #se obtine el vocab
	keywords=dict(getData(palabras.tocoo(),feature_names)) # obtenemos el vocab relacionado con sus tf-idf
	vectores={} 
	cont=0
	total=len(corpus)
	ti=time.time()
	for c in corpus: #para cada documentos c en el corpus
		vectores[c[0]]=doc2Vec(c[1],keywords,funcion) # se transforma a un vector
		print("llevo ",cont," de ",total,end="\r") # contamos cuantos lleva
		cont+=1
	print("Me tarde : ",time.time()-ti)
	pickle.dump([vectores,keywords],open(archPickle,"wb")) #lo guardamos en un pickle para despues
	return [vectores,keywords] #regresamos los vectores y el vocab
funcion=lambda x:1/(1+exp(5*(1-x))) #Función sigmoide para tener los valores en un rango 0 y 1

vectores,vocab=crearVectores("datosLimpios.csv","vectoresExp4.pkl",funcion)
print(len(vocab))
#esto suele tardar 48 segundos

/home/max/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmo', 'agrego', 'algun', 'anadio', 'aseguro', 'comento', 'considero', 'dejo', 'demas', 'estabamos', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'explico', 'expreso', 'fueramos', 'fuesemos', 'habeis', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'indico', 'llego', 'manifesto', 'menciono', 'ningun', 'podra', 'podran', 'proximos', 'quedo', 'realizo', 'seais', 'senalo', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria

Me tarde :  48.80319881439209
4538


Una vez que tenemos los vectores de los documentos, es necesario comparar cada uno con el vector que describe nuestros síntomas

In [5]:
from scipy.spatial import distance

def predecir(archSintomas,vectores,vocab,funcion):
	sintomas=open(archSintomas,"r").read()
	vector=doc2Vec(sintomas,vocab,funcion)
	docs=[]
	for n,v in vectores.items():
		# docs.append([n,distance.cdist( vector,v, 'wminkowski',w=np.random.rand(v.shape[0]))])
		# docs.append([n,distance.cdist( vector,v, 'matching')])
		# docs.append([n,distance.cdist( vector,v, 'braycurtis')])
		# docs.append([n,distance.cdist( vector,v, 'canberra')])
		# docs.append([n,distance.cdist( vector,v, 'chebyshev')])
		# docs.append([n,distance.cdist( vector,v, 'jaccard')])
		# docs.append([n,distance.cdist( vector,v, 'correlation')])
		# docs.append([n,distance.cdist( vector,v, 'sqeuclidean')])
		# docs.append([n,distance.cdist( vector,v, 'cityblock')])
		docs.append([n,distance.cdist( vector,v, 'cosine')]) #es la que mejor sirve
		# docs.append([n,distance.cdist( vector,v, 'euclidean')])
	docs.sort(key=lambda x:x[1])
	return docs
dists=predecir("misSintomas.txt",vectores,vocab,funcion)

Ya por último, tenemos que mostrar el nombre de las enfermedades más cercanas

In [6]:
for i in range(10):
	print(dists[i])

['espondilitis_anquilosante.html ', array([[0.55964787]])]
['sacralgia.html ', array([[0.57724832]])]
['dolor_espalda.html ', array([[0.58849898]])]
['hipermetropia.html ', array([[0.63843594]])]
['ciatica.html ', array([[0.67218111]])]
['enfermedad_sudeck.html ', array([[0.68650639]])]
['hiperparatiroidismo.html ', array([[0.68776322]])]
['cefalea_en_racimos.html ', array([[0.69026291]])]
['dismenorrea.html ', array([[0.70119622]])]
['epididimitis.html ', array([[0.71563155]])]


# Conclusiones
Como conclusión este sistema funciona relativamente bien, ya que no siempre regresa las enfermedades que son más intuitivas para un médico, este sistema no sabe que hay enfermedades comunes y otras que no, también realicé otro experimento donde encuentro los tf-idf por sin usar tdfvectorizer, ver Experimento5

In [7]:
dists=predecir("misSintomas2.txt",vectores,vocab,funcion) #sintomas alergia
for i in range(10):
    print(dists[i])

['cancer-cabeza-cuello.html ', array([[0.78392699]])]
['ronquido.html ', array([[0.79196845]])]
['piedras_en_la_vesicula.html ', array([[0.80235308]])]
['leucemia.html ', array([[0.80323531]])]
['apnea_del_sueno.html ', array([[0.80835584]])]
['bruxismo.html ', array([[0.80857495]])]
['leucoplasia.html ', array([[0.81379813]])]
['arteriosclerosis.html ', array([[0.81670861]])]
['enfermedad_mano_pie_boca.html ', array([[0.82025684]])]
['quiste_ovario.html ', array([[0.82574736]])]


In [9]:
dists=predecir("misSintomas3.txt",vectores,vocab,funcion) #sintomas gripe 
for i in range(10):
    print(dists[i])

['hirsutismo.html ', array([[0.73576919]])]
['tenosinovitis.html ', array([[0.76819148]])]
['sudor_de_manos.html ', array([[0.80680944]])]
['cirrosis_hepatica.html ', array([[0.81389452]])]
['diarrea.html ', array([[0.81656632]])]
['hiperhidrosis.html ', array([[0.81729501]])]
['polipos_intestinales.html ', array([[0.81901033]])]
['narcolepsia.html ', array([[0.82030984]])]
['apnea_del_sueno.html ', array([[0.82505592]])]
['sordera_subita.html ', array([[0.82709366]])]


In [10]:
dists=predecir("prueba.txt",vectores,vocab,funcion) #sintomas gripe de otra pagina
for i in range(10):
    print(dists[i])

['gripe_porcina.html ', array([[0.47269598]])]
['resfriado.html ', array([[0.50981528]])]
['legionela.html ', array([[0.58280845]])]
['malaria.html ', array([[0.5900416]])]
['gripe.html ', array([[0.63739934]])]
['bronquiectasia.html ', array([[0.68786715]])]
['miocarditis.html ', array([[0.69214178]])]
['sarampion.html ', array([[0.7146384]])]
['bronquiolitis.html ', array([[0.72048584]])]
['escarlatina.html ', array([[0.72448795]])]


Como conclusión podemos decir que el sistema funciona relativamente bien, ya que muestra enfermedades con sintomas parecidos, aunque no prioriza la enfermedades mas probables, y con una buena descripción de los sintomas funciona bien.